
# Train on Jan–Nov 2023, Predict Dec 2023 (Power LSTM)

This notebook:
1. Trains an LSTM model on **January–November 2023** data.
2. Validates with the tail of that training window (chronological split).
3. Generates predictions for **December 2023** and computes metrics.
4. Saves artifacts (model + scalers + config) and CSV outputs.
5. Parses the CSV results, visualizes and plots the CSV outputs.



In [1]:

import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import r2_score
import joblib
import matplotlib.dates as mdates



In [2]:

# ======================
# Configuration
# ======================

# Site / data location
#site = 'S0186'
#postcode = 5290

#site = 'S0561'
#postcode = 5555

#site = 'S0150'
#postcode = 5035

site = 'S0556'
postcode = 5098

data_directory = '../Data/Processed'
site_filepath = os.path.join(data_directory, f"SA_site_edp_2023_{site}_processed.csv")
nci_path = "../Data/NCI/NCI_processed_grouped_all_SA_2023.csv"

# Time windows (local Australia/Adelaide)
tz = "Australia/Adelaide"
TRAIN_START = pd.Timestamp('2023-01-01 00:00:00', tz=tz)
TRAIN_END   = pd.Timestamp('2023-11-30 23:59:59', tz=tz)
PRED_START  = pd.Timestamp('2023-12-01 00:00:00', tz=tz)
PRED_END    = pd.Timestamp('2023-12-31 23:59:59', tz=tz)

# Modeling params

# lookback length
timesteps = 6                  
merge_tolerance = "10min"

# validation fraction from the *end* of the training window
val_frac = 0.1                 

# include 'Power' in inputs
use_past_power = True

# train only on daylight sequences
mask_daylight_in_train = True  
use_solar_elevation_for_day = True

# used if use_solar_elevation_for_day=False
ghi_day_threshold = 20.0       

# Artifacts/output
artifacts_dir = "./artifacts_power_lstm_2023_11train"
os.makedirs(artifacts_dir, exist_ok=True)
# Keras 3 format
model_path = os.path.join(artifacts_dir, "best_lstm_power.keras")   
scaler_feat_path = os.path.join(artifacts_dir, "scaler_features.gz")
# only used when use_past_power=False
scaler_tgt_path  = os.path.join(artifacts_dir, "scaler_target.gz")  
config_path = os.path.join(artifacts_dir, "config.json")

pred_csv_path = os.path.join(artifacts_dir, "power_predictions_2023_12.csv")
daily_metrics_csv_path = os.path.join(artifacts_dir, "power_daily_metrics_2023_12.csv")


In [3]:

# ======================
# Helpers
# ======================
def localize_adelaide_naive(series: pd.Series) -> pd.Series:
    """Localize naive timestamps to Australia/Adelaide with DST handling."""
    return series.dt.tz_localize(
        "Australia/Adelaide",
        ambiguous="NaT",
        nonexistent="shift_forward"
    )

def align_site_nci(site_filepath: str,
                   nci_path: str,
                   postcode: int,
                   start_time: pd.Timestamp,
                   end_time: pd.Timestamp,
                   merge_tolerance: str = "10min") -> pd.DataFrame:
    """Load site + NCI, align on time in Australia/Adelaide, return merged frame."""
    # Site
    df_site = pd.read_csv(site_filepath)
    if 'Power' not in df_site.columns:
        raise KeyError("Expected 'Power' column in site CSV.")
    df_site['datetime'] = pd.to_datetime(df_site['datetime'], errors='coerce')
    if df_site['datetime'].dt.tz is None:
        df_site['datetime'] = localize_adelaide_naive(df_site['datetime'])
    else:
        df_site['datetime'] = df_site['datetime'].dt.tz_convert('Australia/Adelaide')
    df_site = df_site[(df_site['datetime'] >= start_time) & (df_site['datetime'] <= end_time)].copy()
    if df_site.empty:
        raise ValueError("No site data in the selected time window after timezone alignment.")
    df_site = df_site.sort_values('datetime')

    # NCI
    nci = pd.read_csv(nci_path)
    nci = nci[nci['postcode'].notna()].copy()
    nci['postcode'] = nci['postcode'].astype(int)
    nci = nci[nci['postcode'] == postcode].copy()
    if nci.empty:
        raise ValueError(f"No NCI rows for postcode {postcode} in {nci_path}")
    nci['time'] = pd.to_datetime(nci['time'], errors='coerce', utc=True).dt.tz_convert('Australia/Adelaide')

    # Derive GHI if needed
    if 'surface_global_irradiance' not in nci.columns:
        req = {'direct_normal_irradiance','surface_diffuse_irradiance','solar_elevation'}
        if not req.issubset(nci.columns):
            raise KeyError("NCI missing 'surface_global_irradiance' and components to derive it.")
        nci['zenith_angle'] = 90 - nci['solar_elevation']
        nci['surface_global_irradiance'] = (
            nci['surface_diffuse_irradiance']
            + nci['direct_normal_irradiance'] * np.cos(np.radians(nci['zenith_angle']))
        )

    keep = [
        'time',
        'surface_global_irradiance',
        'direct_normal_irradiance',
        'surface_diffuse_irradiance',
        'cloud_type',
        'cloud_optical_depth',
        'solar_elevation',
        'solar_azimuth',
    ]
    for c in keep:
        if c not in nci.columns:
            raise KeyError(f"NCI missing required column: {c}")

    nci_small = nci[keep].sort_values('time')

    # Align (nearest within tolerance)
    merged = pd.merge_asof(
        df_site, nci_small,
        left_on='datetime', right_on='time',
        direction='nearest', tolerance=pd.Timedelta(merge_tolerance)
    ).drop(columns=['time'])

    # Interpolate NCI cols
    for c in ['surface_global_irradiance','direct_normal_irradiance','surface_diffuse_irradiance',
              'cloud_type','cloud_optical_depth','solar_elevation','solar_azimuth']:
        merged[c] = merged[c].interpolate(limit_direction='both')

    return merged

def create_sequences(data2d: np.ndarray, steps: int, target_col: int):
    """Return X, y, row_idx for one-step-ahead sequences."""
    X, y, idx = [], [], []
    for i in range(steps, len(data2d)):
        X.append(data2d[i-steps:i, :])
        y.append(data2d[i, target_col])
        idx.append(i)
    return np.array(X), np.array(y).reshape(-1,1), np.array(idx)

def make_sequences_for_inference(features_scaled: np.ndarray, timesteps: int):
    X, row_idx = [], []
    for i in range(timesteps, len(features_scaled)):
        X.append(features_scaled[i-timesteps:i, :])
        row_idx.append(i)
    return np.array(X), np.array(row_idx)

def inverse_minmax_column(pred_scaled: np.ndarray, scaler: MinMaxScaler, col_idx: int) -> np.ndarray:
    data_min = scaler.data_min_[col_idx]
    data_max = scaler.data_max_[col_idx]
    return pred_scaled * (data_max - data_min) + data_min


In [4]:

# ======================
# Load & prepare TRAIN (Jan–Nov 2023)
# ======================
merged_train = align_site_nci(
    site_filepath=site_filepath,
    nci_path=nci_path,
    postcode=postcode,
    start_time=TRAIN_START,
    end_time=TRAIN_END,
    merge_tolerance=merge_tolerance
)

# Feature columns
if use_past_power:
    feature_cols = [
        'direct_normal_irradiance',
        'surface_diffuse_irradiance',
        'cloud_type',
        'cloud_optical_depth',
        'solar_elevation',
        'solar_azimuth',
        'surface_global_irradiance',
        'Power'
    ]
else:
    feature_cols = [
        'direct_normal_irradiance',
        'surface_diffuse_irradiance',
        'cloud_type',
        'cloud_optical_depth',
        'solar_elevation',
        'solar_azimuth',
        'surface_global_irradiance'
    ]

target_name = 'Power'
for c in feature_cols + [target_name]:
    if c not in merged_train.columns:
        raise KeyError(f"Missing column in training data: {c}")

# Raw arrays
features_raw_train = merged_train[feature_cols].ffill().bfill().to_numpy(dtype=np.float64)
target_raw_train   = merged_train[[target_name]].ffill().bfill().to_numpy(dtype=np.float64)

# Chronological split inside Jan–Nov block
N_tr = len(features_raw_train)
split_row = int(N_tr * (1 - val_frac))
if split_row <= timesteps:
    raise ValueError("Not enough rows in training for chosen timesteps/val split.")

# Scale features (fit on train-only slice)
scaler_features = MinMaxScaler()
scaler_features.fit(features_raw_train[:split_row])
features_scaled_train = scaler_features.transform(features_raw_train)

# Target scaling (if target not included in features)
if use_past_power:
    target_in_features = True
    target_col_idx = feature_cols.index(target_name)
    scaler_target = None
else:
    target_in_features = False
    target_col_idx = None
    scaler_target = MinMaxScaler()
    scaler_target.fit(target_raw_train[:split_row])
    target_scaled_train = scaler_target.transform(target_raw_train)

# Sequences across whole Jan–Nov block
if use_past_power:
    X_all_tr, y_all_tr, row_idx_tr = create_sequences(features_scaled_train, timesteps, target_col_idx)
else:
    # dummy to get row_idx, y from target_scaled
    X_all_tr, _, row_idx_tr = create_sequences(features_scaled_train, timesteps, target_col=0)
    y_all_tr = np.array([target_scaled_train[i, 0] for i in row_idx_tr]).reshape(-1, 1)

# Daylight mask for TRAIN only
if mask_daylight_in_train:
    if use_solar_elevation_for_day:
        daylight_row = (merged_train['solar_elevation'].to_numpy() > 0)
    else:
        daylight_row = (merged_train['surface_global_irradiance'].to_numpy() > ghi_day_threshold)

    day_series = pd.Series(daylight_row.astype(int))
    all_inputs_day = (
        day_series.shift(1)  # window ends at i-1
                 .rolling(window=timesteps, min_periods=timesteps)
                 .min()
                 .fillna(0)
                 .astype(bool)
                 .to_numpy()
    )
    target_is_day = daylight_row
    sample_ok = target_is_day & all_inputs_day
    seq_ok_tr = sample_ok[row_idx_tr]
else:
    seq_ok_tr = np.ones_like(row_idx_tr, dtype=bool)

# Split into train/val (chronological)
train_mask_time = (row_idx_tr < split_row)
val_mask_time   = (row_idx_tr >= split_row)
train_mask = train_mask_time & seq_ok_tr
val_mask   = val_mask_time   # keep evening/night in validation to measure robustness

X_train, y_train = X_all_tr[train_mask], y_all_tr[train_mask]
X_val,   y_val   = X_all_tr[val_mask],   y_all_tr[val_mask]

X_train = X_train.astype(np.float32); y_train = y_train.astype(np.float32)
X_val   = X_val.astype(np.float32);   y_val   = y_val.astype(np.float32)

print(f"Jan–Nov rows: {N_tr:,}")
print(f"Train samples: {X_train.shape[0]}  (kept {train_mask.sum()} / {train_mask_time.sum()} after daylight mask)")
print(f"Val   samples: {X_val.shape[0]}    (kept {val_mask.sum()} / {val_mask_time.sum()})")


Jan–Nov rows: 96,149
Train samples: 61358  (kept 61358 / 86528 after daylight mask)
Val   samples: 9615    (kept 9615 / 9615)


In [5]:

# ======================
# Build datasets & train model
# ======================
batch_size = 512
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val,   y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

n_features = X_train.shape[-1]
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(timesteps, n_features)),
    Dropout(0.2),
    LSTM(32),
    Dense(1),
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MeanSquaredError())

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path, save_best_only=True)  # saves .keras

history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[early_stop, checkpoint])

# Save scalers + config
joblib.dump(scaler_features, scaler_feat_path)
if not target_in_features:
    joblib.dump(scaler_target, scaler_tgt_path)

config = {
    "timesteps": timesteps,
    "feature_cols": feature_cols,
    "target_name": "Power",
    "target_in_features": target_in_features,
    "target_col_idx": target_col_idx,
    "merge_tolerance": merge_tolerance,
    "mask_daylight_in_train": mask_daylight_in_train,
    "use_solar_elevation_for_day": use_solar_elevation_for_day,
    "ghi_day_threshold": ghi_day_threshold,
    "train_window": [str(TRAIN_START), str(TRAIN_END)],
    "predict_window": [str(PRED_START), str(PRED_END)],
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)

print("Saved artifacts to:", artifacts_dir)


Epoch 1/20


C:\Users\Monica\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0156 - val_loss: 0.0035
Epoch 2/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0041 - val_loss: 0.0031
Epoch 3/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0038 - val_loss: 0.0028
Epoch 4/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0037 - val_loss: 0.0027
Epoch 5/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0036 - val_loss: 0.0027
Epoch 6/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0035 - val_loss: 0.0027
Epoch 7/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 8/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0033 - val_loss: 0.0027
Epoch 9/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 10/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 11/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0030 - val_loss: 0.0025
Epoch 12/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/ste

In [6]:
# ==============================
# Metrics (all times + daylight-only)
# ==============================
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# ---- Extra metrics ----
def mse(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return float(np.mean((y_pred - y_true) ** 2))

def mape(y_true, y_pred, ignore_zeros=True, eps=1e-8):
    """
    MAPE in %, by default ignores y_true == 0 (common at night for PV).
    """
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.abs(y_true)
    if ignore_zeros:
        m = denom > 0
        y_true, y_pred, denom = y_true[m], y_pred[m], denom[m]
    denom = np.maximum(denom, eps)
    if denom.size == 0:
        return float("nan")
    return float(np.mean(np.abs((y_pred - y_true) / denom)) * 100.0)

def wape(y_true, y_pred):
    """
    WAPE in % = sum|e| / sum|y_true|.
    """
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return float("nan")
    return float(np.sum(np.abs(y_pred - y_true)) / denom * 100.0)

def _infer_seasonal_periods_from_times(dt_series: pd.Series) -> int:
    """
    Infer seasonal period (samples per day) from timestamps.
    Falls back to 1 if inference fails.
    """
    try:
        s = pd.to_datetime(pd.Series(dt_series)).sort_values()
        if s.size < 3:
            return 1
        step = s.diff().dropna().median()
        if pd.isna(step) or step <= pd.Timedelta(0):
            return 1
        sp = int(round(pd.Timedelta(days=1) / step))
        return max(1, sp)
    except Exception:
        return 1

def mase(y_true, y_pred, y_train, seasonal_periods=1):
    """
    MASE per Hyndman & Koehler (2006).
    Scale = MAE of seasonal naive forecast on TRAIN actuals:
        mean(|y_t - y_{t-m}|), t=m..T-1
    Returns NaN if scale cannot be computed.
    """
    if y_train is None:
        return float("nan")
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    y_train = np.asarray(y_train).reshape(-1)
    if y_train.size <= seasonal_periods:
        return float("nan")
    naive_err = np.abs(y_train[seasonal_periods:] - y_train[:-seasonal_periods])
    scale = np.mean(naive_err[np.isfinite(naive_err)])
    if not np.isfinite(scale) or scale == 0:
        return float("nan")
    return float(np.mean(np.abs(y_pred - y_true)) / scale)





# ======================
# Predict December 2023
# ======================
# Reload model (best checkpoint)
model = tf.keras.models.load_model(model_path)

merged_pred = align_site_nci(
    site_filepath=site_filepath,
    nci_path=nci_path,
    postcode=postcode,
    start_time=PRED_START,
    end_time=PRED_END,
    merge_tolerance=merge_tolerance
)

# Check columns
for c in feature_cols + ["Power"]:
    if c not in merged_pred.columns:
        raise KeyError(f"Missing column in prediction data: {c}")

# Scale using training scaler
features_raw_pred = merged_pred[feature_cols].ffill().bfill().to_numpy(dtype=np.float64)
features_scaled_pred = scaler_features.transform(features_raw_pred)

# Build sequences for inference (all rows, incl. night)
def make_sequences_for_inference(features_scaled: np.ndarray, timesteps: int):
    X, row_idx = [], []
    for i in range(timesteps, len(features_scaled)):
        X.append(features_scaled[i-timesteps:i, :])
        row_idx.append(i)
    return np.array(X), np.array(row_idx)

X_inf, row_idx = make_sequences_for_inference(features_scaled_pred, timesteps)
X_inf = X_inf.astype(np.float32)

# Predict (scaled)
batch_size_inf = 2048
inf_ds = tf.data.Dataset.from_tensor_slices(X_inf).batch(batch_size_inf).prefetch(tf.data.AUTOTUNE)
pred_scaled = model.predict(inf_ds, verbose=0).reshape(-1)

# Inverse-scale to file units
def inverse_minmax_column(pred_scaled: np.ndarray, scaler: MinMaxScaler, col_idx: int) -> np.ndarray:
    data_min = scaler.data_min_[col_idx]
    data_max = scaler.data_max_[col_idx]
    return pred_scaled * (data_max - data_min) + data_min

if target_in_features:
    power_pred = inverse_minmax_column(pred_scaled, scaler_features, col_idx=target_col_idx)
else:
    if os.path.exists(scaler_tgt_path):
        scaler_target = joblib.load(scaler_tgt_path)
        power_pred = scaler_target.inverse_transform(pred_scaled.reshape(-1,1)).reshape(-1)
    else:
        raise RuntimeError("Target scaler not found for non-autoregressive model.")

# Assemble results
times_all = merged_pred['datetime'].reset_index(drop=True)
times_pred = times_all.iloc[row_idx].reset_index(drop=True)
results = pd.DataFrame({
    "datetime": times_pred,
    "Power_pred_W": power_pred,
    "Power_true_W": merged_pred['Power'].to_numpy()[row_idx]
})




# Try to supply training ACTUALS for MASE if available.
# Set one of these earlier in your notebook to make MASE work:
#   y_train_actuals = <1D array/Series of training-period actual Power in W>
# Otherwise MASE will return NaN (safe fallback).
y_train_for_mase = None
for _cand in ["y_train_actuals", "y_train_power", "train_target", "train_y", "y_train"]:
    if _cand in globals():
        y_train_for_mase = globals()[_cand]
        break

seasonal_periods = _infer_seasonal_periods_from_times(results["datetime"])

def _metrics_full(y_t, y_p, y_train_for_mase, seasonal_periods):
    y_t = np.asarray(y_t); y_p = np.asarray(y_p)
    mae  = float(np.mean(np.abs(y_p - y_t)))
    _mse = mse(y_t, y_p)
    rmse = float(np.sqrt(_mse))
    r2   = float(r2_score(y_t, y_p)) if len(y_t) > 1 else float("nan")
    _wape = wape(y_t, y_p)
    _mase = mase(y_t, y_p, y_train_for_mase, seasonal_periods)
    return {
        "MAE": mae,
        "MSE": _mse,
        "RMSE": rmse,
        "R2": r2,
        "WAPE_%": _wape,
        "N": int(len(y_t))
    }

y_true = results["Power_true_W"].to_numpy()
y_pred = results["Power_pred_W"].to_numpy()

metrics_all = _metrics_full(y_true, y_pred, y_train_for_mase, seasonal_periods)

solar_elev_seq = merged_pred['solar_elevation'].to_numpy()[row_idx]
day_mask = solar_elev_seq > 0
if day_mask.any():
    metrics_day = _metrics_full(y_true[day_mask], y_pred[day_mask], y_train_for_mase, seasonal_periods)
else:
    metrics_day = {k: float("nan") for k in ["MAE","MSE","RMSE","R2","MAPE_%","WAPE_%","MASE"]}
    metrics_day["N"] = 0

# Pretty print
print("December 2023 metrics")
print(
    "DAYLIGHT AND NIGHT times -> "
    f"MAE: {metrics_all['MAE']:,.6f}  "
    f"MSE: {metrics_all['MSE']:,.6f}  "
    f"RMSE: {metrics_all['RMSE']:,.6f}  "
    f"WAPE: {metrics_all['WAPE_%']:,.3f}%  "
    f"R²: {metrics_all['R2']:,.4f}  "
    f"N: {metrics_all['N']}"
)
print(
    "DAYLIGHT ONLY -> "
    f"MAE: {metrics_day['MAE']:,.6f}  "
    f"MSE: {metrics_day['MSE']:,.6f}  "
    f"RMSE: {metrics_day['RMSE']:,.6f}  "
    f"WAPE: {metrics_day['WAPE_%']:,.3f}%  "
    f"R²: {metrics_day['R2']:,.4f}  "
    f"N: {metrics_day['N']}"
)

# ==============================
# Save outputs (unchanged)
# ==============================
results.to_csv(pred_csv_path, index=False)

# ==============================
# Daily metrics (expanded)
# ==============================
results['date'] = results['datetime'].dt.date
results['daylight'] = day_mask

def _agg_all(g):
    stats = _metrics_full(
        g['Power_true_W'].to_numpy(),
        g['Power_pred_W'].to_numpy(),
        y_train_for_mase,
        seasonal_periods
    )
    return pd.Series({
        'MAE': stats['MAE'],
        'MSE': stats['MSE'],
        'RMSE': stats['RMSE'],
        'WAPE_%': stats['WAPE_%'],
        'R2': stats['R2'],
        'N': stats['N'],
    })

def _agg_day(g):
    mask = g['daylight'].to_numpy()
    if mask.any():
        stats = _metrics_full(
            g.loc[mask, 'Power_true_W'].to_numpy(),
            g.loc[mask, 'Power_pred_W'].to_numpy(),
            y_train_for_mase,
            seasonal_periods
        )
        return pd.Series({
            'MAE_day': stats['MAE'],
            'MSE_day': stats['MSE'],
            'RMSE_day': stats['RMSE'],
            'WAPE_day_%': stats['WAPE_%'],
            'R2_day': stats['R2'],
            'N_day': stats['N'],
        })
    else:
        return pd.Series({
            'MAE_day': float("nan"),
            'MSE_day': float("nan"),
            'RMSE_day': float("nan"),
            'WAPE_day_%': float("nan"),
            'R2_day': float("nan"),
            'N_day': 0
        })

daily_all = results.groupby('date', sort=True).apply(_agg_all).reset_index()
daily_day = results.groupby('date', sort=True).apply(_agg_day).reset_index()
daily_metrics = pd.merge(daily_all, daily_day, on='date', how='left')
daily_metrics.to_csv(daily_metrics_csv_path, index=False)

#print("\nSaved:")
#print("  Predictions ->", pred_csv_path)
#print("  Daily metrics ->", daily_metrics_csv_path)


December 2023 metrics
DAYLIGHT AND NIGHT times -> MAE: 0.143905  MSE: 0.078748  RMSE: 0.280620  WAPE: 14.661%  R²: 0.9522  N: 8917
DAYLIGHT ONLY -> MAE: 0.198396  MSE: 0.111786  RMSE: 0.334344  WAPE: 14.236%  R²: 0.9367  N: 6275


C:\Users\Monica\AppData\Local\Temp\ipykernel_15932\466794745.py:261: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_all = results.groupby('date', sort=True).apply(_agg_all).reset_index()
C:\Users\Monica\AppData\Local\Temp\ipykernel_15932\466794745.py:262: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_day = results.groupby('date', sort=True).apply(_agg_day).reset_index()


# <span style="color:green"> Prediction Accuracy Metrics </span>

1. **Mean Absolute Error (MAE)**
- **Definition:** Average absolute difference between predicted and actual values.
- <span style="color:green"> **Good prediction:** </span> Small MAE (close to 0) → on average, predictions deviate only slightly from actual power.
- <span style="color:red"> **Bad prediction:** </span> Large MAE (no upper bound) → predictions consistently miss the target by a wide margin.
- **Scale:** Interpreted in Watts


2. **Mean Squared Error (MSE)**
- **Definition:** Average of squared differences between predicted and actual values. Penalizes larger errors more than smaller ones.
- <span style="color:green"> **Good prediction:** </span> Low MSE → few large deviations.
- <span style="color:red"> **Bad prediction:** </span> High MSE → presence of large outliers/errors.
- **Scale:** In squared units (Watts²), so less intuitive than MAE.

3. Root Mean Squared Error (RMSE)

- **Definition:** Square root of MSE; same unit as target variable.
- <span style="color:green"> **Good prediction:** </span> Low RMSE → model tracks actual data well with few large errors.
- <span style="color:red"> **Bad prediction:** </span> High RMSE → predictions have large swings away from truth.
- **Interpretation:** RMSE ≥ MAE usually; if RMSE is much larger than MAE, it means some large outliers dominate error.



4. Weighted Absolute Percentage Error (WAPE)

- **Definition:** Total absolute error divided by total actuals, expressed as a percentage.
- <span style="color:green"> **Good prediction:** </span> WAPE close to 0%.
Rule of thumb thresholds:
    - <10% → very good
    - 10–20% → acceptable
    - 20–50% → weak but sometimes tolerable
    - 50% or above → poor
- <span style="color:red"> **Bad prediction:** </span> High WAPE suggests systematic deviation when compared to the magnitude of total production.


5. **R² (R-squared, Coefficient of Determination)**
- **Definition:** Proportion of variance in the actual data explained by the predictions (0–1 scale, sometimes negative if worse than a baseline).
- <span style="color:green"> **Good prediction:** </span> 
    - R² close to 1.0 → model explains almost all variability in actual power.
    - R² ≥ 0.9 → excellent; 
    - R² = 0.7–0.9 → good.
- <span style="color:red"> **Bad prediction:** </span> 
    - R² close to 0 → model does not explain variability.
    - Negative R² → model performs worse than simply predicting the mean of actuals.
**Scale:** Dimensionless, bounded between (−∞, 1].

# <span style="color:green"> Summary - Site X </span>
Errors are slightly higher during daylight (MAE_day ≈ 0.19 vs. MAE ≈ 0.14).
This makes sense: at night, power is zero and trivial to predict, while daylight introduces variability from weather/clouds.

- MAE & MSE & RMSE are low → predictions are generally close in absolute terms.
- WAPE ~18% → forecasts are moderately accurate; good but not best-in-class.
- High R² (0.9+) → strong explanatory power, capturing most variability.


In [7]:
# =========================================
# Plot ALL December 2023 daily Predicted vs Actual power
# + polyfit line (06:00–20:00 local)  + GHI overlay (right y-axis)
# And an extra plot per day: Power (left) + Voltage (right)
# Clamp Power series at 0 (no negatives); Voltage unaltered
# =========================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# ---- Paths (use the one from earlier cell if available) ----
try:
    csv_path = pred_csv_path
except NameError:
    csv_path = "./artifacts_power_lstm_2023_11train/power_predictions_2023_12.csv"

try:
    data_directory
except NameError:
    data_directory = '../Data/Processed'
try:
    nci_path
except NameError:
    nci_path = "../Data/NCI/NCI_processed_grouped_all_SA_2023.csv"

site_filepath = os.path.join(data_directory, f"SA_site_edp_2023_{site}_processed.csv")

# allow site to be interpolated in folder name if it's already defined
try:
    output_dir = f"./december_daily_plots_{site}_Polyfit"
except NameError:
    output_dir = "./december_daily_plots_Polyfit"

pdf_path = os.path.join(output_dir, "power_december_2023_plots.pdf")
show_plots = False  # set True to display as they are created

# Polyfit options
poly_degree = 2
tz = "Australia/Adelaide"
poly_start_h = 6    # 06:00 (inclusive)
poly_end_h   = 20   # 20:00 (inclusive)

# ---- Helper: robust timezone handling ----
def to_adelaide_series_any(df: pd.DataFrame, col: str) -> pd.Series:
    s = pd.to_datetime(df[col], errors="coerce", utc=True)
    if s.notna().any():
        return s.dt.tz_convert(tz)
    s = pd.to_datetime(df[col], errors="coerce")
    if s.isna().all():
        raise ValueError(f"Could not parse any datetimes from '{col}'.")
    tzinfo = getattr(s.dtype, "tz", None)
    if tzinfo is None:
        return s.dt.tz_localize(tz)
    return s.dt.tz_convert(tz)

# ---- Load predictions ----
df = pd.read_csv(csv_path)
if "datetime" not in df.columns:
    raise KeyError("Expected 'datetime' column in predictions CSV.")
if "Power_pred_W" not in df.columns:
    raise KeyError("Expected 'Power_pred_W' column in predictions CSV.")

df["datetime"] = to_adelaide_series_any(df, "datetime")

# ---- Filter to Dec 2023 (inclusive) ----
start_ts = pd.Timestamp("2023-12-01 00:00:00", tz=tz)
end_ts   = pd.Timestamp("2023-12-31 23:59:59", tz=tz)
mask = (df["datetime"] >= start_ts) & (df["datetime"] <= end_ts)
dfr = df.loc[mask].copy().sort_values("datetime")
if dfr.empty:
    raise ValueError("No rows found for December 2023 in the predictions file.")

# ---- Pull GHI + site data for the same window using your existing helper ----
# Requires align_site_nci() to be defined in the notebook/session
dec_merged_for_ghi = align_site_nci(
    site_filepath=site_filepath,
    nci_path=nci_path,
    postcode=postcode,
    start_time=start_ts,
    end_time=end_ts,
    merge_tolerance="10min"
)

# Sanity check for voltage column presence
if 'voltage_avg' not in dec_merged_for_ghi.columns:
    raise KeyError("Expected 'voltage_avg' column in the site dataset.")

ghi_df = dec_merged_for_ghi[["datetime", "surface_global_irradiance"]].copy()
ghi_df = ghi_df.sort_values("datetime").reset_index(drop=True)

# Also keep voltage from the same merged frame
volt_df = dec_merged_for_ghi[["datetime", "voltage_avg"]].copy().sort_values("datetime").reset_index(drop=True)

# ---- Plot per day ----
os.makedirs(output_dir, exist_ok=True)
has_actual = "Power_true_W" in dfr.columns

# add a local calendar date column for grouping
dfr["date"] = dfr["datetime"].dt.date
ghi_df["date"] = ghi_df["datetime"].dt.date
volt_df["date"] = volt_df["datetime"].dt.date

with PdfPages(pdf_path) as pdf:
    for date_value, g in dfr.groupby("date", sort=True):
        g = g.sort_values("datetime")

        # choose which series to fit (prefer actual if we have it)
        y_fit_col = "Power_true_W" if has_actual else "Power_pred_W"

        # --- define the polyfit window: poly_start_h to poly_end_h local ---
        day_start = pd.Timestamp(f"{date_value} {poly_start_h:02d}:00:00", tz=tz)
        day_end   = pd.Timestamp(f"{date_value} {poly_end_h:02d}:00:00",   tz=tz)

        gp = g[(g["datetime"] >= day_start) & (g["datetime"] <= day_end)].copy()

        # prepare x as seconds since poly_start_h (helps conditioning)
        t0 = day_start
        if not gp.empty:
            x_fit = (gp["datetime"] - t0).dt.total_seconds().to_numpy()
            y_fit = gp[y_fit_col].to_numpy()
        else:
            x_fit = np.array([])
            y_fit = np.array([])

        # drop NaNs for fitting
        valid = np.isfinite(x_fit) & np.isfinite(y_fit)
        poly_added = False
        y_poly_eval = None

        if valid.sum() >= (poly_degree + 1):
            try:
                coeffs = np.polyfit(x_fit[valid], y_fit[valid], deg=poly_degree)
                seconds_span = int((day_end - day_start).total_seconds())
                x_eval = np.linspace(0, seconds_span, 721)  # ~one-minute resolution
                y_poly_eval = np.poly1d(coeffs)(x_eval)
                y_poly_eval = np.clip(y_poly_eval, 0, None)  # clamp for plotting
                dt_eval = day_start + pd.to_timedelta(x_eval, unit="s")
                poly_added = True
            except Exception:
                poly_added = False

        # --- slice GHI and Voltage for the same day (full day) ---
        ghi_day = ghi_df.loc[ghi_df["date"] == date_value, ["datetime", "surface_global_irradiance"]]
        has_ghi = not ghi_day.empty and ghi_day["surface_global_irradiance"].notna().any()
        if has_ghi:
            ghi_vals = np.clip(ghi_day["surface_global_irradiance"].to_numpy(), 0, None)

        volt_day = volt_df.loc[volt_df["date"] == date_value, ["datetime", "voltage_avg"]]
        has_volt = not volt_day.empty and volt_day["voltage_avg"].notna().any()

        # ---- CLAMP predicted/actual Power for plotting ----
        pred_clamped = np.clip(g["Power_pred_W"].to_numpy(), 0, None)
        if has_actual:
            true_clamped = np.clip(g["Power_true_W"].to_numpy(), 0, None)

        # ========== FIGURE 1: Power (+polyfit) with GHI on right ==========
        fig1, ax1 = plt.subplots()

        l_pred, = ax1.plot(g["datetime"], pred_clamped, label="Predicted Power")
        lines = [l_pred]; labels = ["Predicted Power"]

        if has_actual:
            l_act, = ax1.plot(g["datetime"], true_clamped, label="Actual Power")
            lines.append(l_act); labels.append("Actual Power")

        if poly_added and y_poly_eval is not None:
            l_poly, = ax1.plot(dt_eval, y_poly_eval, linestyle="--", label="Polyfit")
            lines.append(l_poly); labels.append("Polyfit")

        if has_ghi:
            ax2 = ax1.twinx()
            l_ghi, = ax2.plot(ghi_day["datetime"], ghi_vals, linestyle=":", label="GHI")
            ax2.set_ylabel("GHI (W/m²)")
            lines.append(l_ghi); labels.append("GHI")

        ax1.set_title(f"Power & GHI - {date_value} (Australia/Adelaide)")
        ax1.set_xlabel("Time")
        ax1.set_ylabel("Power (Kw)")
        ax1.grid(True)
        ax1.legend(lines, labels, loc="best")
        fig1.autofmt_xdate()
        fig1.tight_layout()

        png_name_1 = os.path.join(output_dir, f"power_ghi_{date_value}.png")
        fig1.savefig(png_name_1, dpi=150)
        pdf.savefig(fig1)
        if show_plots: plt.show()
        plt.close(fig1)

        # ========== FIGURE 2: Power (+polyfit) with Voltage on right ==========
        if has_volt:
            fig2, ax1b = plt.subplots()

            l_pred2, = ax1b.plot(g["datetime"], pred_clamped, label="Predicted Power")
            lines2 = [l_pred2]; labels2 = ["Predicted Power"]

            if has_actual:
                l_act2, = ax1b.plot(g["datetime"], true_clamped, label="Actual Power")
                lines2.append(l_act2); labels2.append("Actual Power")

            if poly_added and y_poly_eval is not None:
                l_poly2, = ax1b.plot(dt_eval, y_poly_eval, linestyle="--", label="Polyfit")
                lines2.append(l_poly2); labels2.append("Polyfit")

            ax2b = ax1b.twinx()
            l_volt, = ax2b.plot(volt_day["datetime"], volt_day["voltage_avg"], linestyle=":", label="Voltage")
            ax2b.set_ylabel("Voltage (V)")
            lines2.append(l_volt); labels2.append("Voltage")

            ax1b.set_title(f"Power & Voltage - {date_value} (Australia/Adelaide)")
            ax1b.set_xlabel("Time")
            ax1b.set_ylabel("Power (Kw)")
            ax1b.grid(True)
            ax1b.legend(lines2, labels2, loc="best")
            fig2.autofmt_xdate()
            fig2.tight_layout()

            png_name_2 = os.path.join(output_dir, f"power_voltage_{date_value}.png")
            fig2.savefig(png_name_2, dpi=150)
            pdf.savefig(fig2)
            if show_plots: plt.show()
            plt.close(fig2)
        else:
            # Optional: log a note if voltage is missing for this day
            print(f"[{date_value}] No voltage data available; skipping Power&Voltage plot.")

print(f"Saved daily PNGs to: {os.path.abspath(output_dir)}")
print(f"Saved combined PDF to: {os.path.abspath(pdf_path)}")


Saved daily PNGs to: c:\Git\NeuralNetwork-EnergyPrediction\NeuralNetwork\december_daily_plots_S0556_Polyfit
Saved combined PDF to: c:\Git\NeuralNetwork-EnergyPrediction\NeuralNetwork\december_daily_plots_S0556_Polyfit\power_december_2023_plots.pdf
